# Section 3 - Average life time 
Questions 
1. What is the average lifetime of an email user (how long do they receive at least one of our email series)? 
1. Does that vary by referrer?


Here, I tried 3 different ways to get the average life time. And I print out the run time of each methods. Finally I choosed function 3 as the most efficient ways to do it. This does not make any difference to my final conclusion. So I didn't put this part in my presentaion slides. Just want to show how I update and enhance a method. 

In [1]:
import mysql.connector
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt  
import datetime

# Use ggplot as plt style
plt.style.use('ggplot') 

In [2]:
cnx = mysql.connector.connect(user='root', password='8xIt7ASAOzIh',
                              host='dw-sandbox-rds.ccia3ag00jrs.us-west-2.rds.amazonaws.com',
                              database='dw')

In [3]:
import time

start_time = time.clock()
query = """
    SELECT created_at,
           baby_registry_101_subscribed_at as baby101,
           newsletter_subscribed_at as news,
           pregnancy_weekly_subscribed_at as preg,
           sponsored_subscribed_at as sponsor,
           IF(baby_registry_101_subscribed_at IS NULL, 0, 1) + IF(newsletter_subscribed_at IS NULL, 0, 1) + IF(pregnancy_weekly_subscribed_at IS NULL, 0, 1) + IF(sponsored_subscribed_at IS NULL, 0, 1) AS DIVERSITY,           
           referrer,
           source
    FROM dim_users 
    WHERE DATE(created_at) > "2017-02-08"
      AND is_active IS NOT NULL
      AND IF(baby_registry_101_subscribed_at IS NULL, 0, 1) + IF(newsletter_subscribed_at IS NULL, 0, 1) + IF(pregnancy_weekly_subscribed_at IS NULL, 0, 1) + IF(sponsored_subscribed_at IS NULL, 0, 1) > 0
"""
df = pd.read_sql_query(query, con = cnx)
print time.clock() - start_time, "seconds"


104.49969 seconds


In [4]:
df.head(3)

,created_at,baby101,news,preg,sponsor,DIVERSITY,referrer,source
0,2017-02-09 00:00:26,2017-02-06 20:20:31,2017-02-24 01:01:58,None,NaT,2,None,checklist
1,2017-02-09 00:02:52,2017-02-11 00:00:46,2017-03-03 01:01:53,None,2017-04-07 18:18:24,3,None,registry
2,2017-02-09 00:03:06,2017-02-11 00:00:53,2017-03-03 01:01:56,None,NaT,2,None,registry


In [5]:
df.shape

(819474, 8)

In [6]:
def is_datetime(data):
    return isinstance(data, datetime.date)
def is_not_nat(data):
    result = type(data) != pd.tslib.NaTType
    return result

### Function 1

In [9]:
def life_time(data):
    lf_time = []
    dt_check = data.applymap(is_datetime)
    for i in range(0, data.shape[0]):
        lf_time.append(min(data.iloc[i, :][dt_check.iloc[i, :]]))
    return lf_time

##### Function 1
Sample : 1000, Time: 0.971258 seconds

Sample: All, Time : 851.704755 seconds

### Function 2

In [ ]:
def first_email(data):
    first_email = []
    for i in range(0, data.shape[0]):
        data_slice = data.iloc[i, :]
        is_date = data_slice.apply(is_datetime)
        data_slice = data_slice[is_date]
        is_value = data_slice.apply(is_not_nat)
        data_slice = data_slice[is_value]
        min_date = min(data_slice[is_date])
        first_email.append(min_date)
    return first_email

In [ ]:
start_time = time.clock()
test_result_2 = first_email(df.iloc[:, 1:5])
print time.clock() - start_time, "seconds"

In [ ]:
test_result_2[-20:]

##### Function 2
Sample : 1000, Time: 1.067037 seconds

Sample: All, Time : 673.597301 seconds

### Function 3

In [9]:
def first_email_each(data_slice):
    first_email = []
    # subset with only datetime
    is_date = data_slice.apply(is_datetime)
    data_slice = data_slice[is_date]
    
    # subset without datetime_nat
    is_value = data_slice.apply(is_not_nat)
    data_slice = data_slice[is_value]
    
    min_date = min(data_slice[is_date])
    first_email.append(min_date)
    return first_email

def first_email(data):
    result = data.apply(first_email_each, axis = 1).iloc[:, 0]
    return result

In [72]:
start_time = time.clock()
test_result_3 = first_email(df.iloc[:30000, 1:5])
print time.clock() - start_time, "seconds"

55.881893 seconds


##### Function 3
Sample : 1000, Time: 1.062652 seconds

Sample: All, Time : 643.059334 seconds

# Average Lifetime

In [57]:
df["First_email"] = test_result_3 

In [58]:
df["Life_time"] = df.First_email.values - df.created_at.values

In [59]:
df.Life_time.describe()

count                           3000
mean     -5404 days +21:42:45.201333
std        7988 days 04:33:29.889349
min            -17211 days +06:15:17
25%            -17207 days +02:28:14
50%           1 days 23:24:02.500000
75%           1 days 23:51:29.500000
max                168 days 20:49:17
Name: Life_time, dtype: object

In [60]:
df.head(3)

,created_at,baby101,news,preg,sponsor,DIVERSITY,referrer,source,First_email,Life_time
0,2017-02-09 00:00:26,2017-02-06 20:20:31,2017-02-24 01:01:58,0,1970-01-01 00:00:00,2,0,checklist,1970-01-01 00:00:00,-17207 days +23:59:34
1,2017-02-09 00:02:52,2017-02-11 00:00:46,2017-03-03 01:01:53,0,2017-04-07 18:18:24,3,0,registry,2017-02-11 00:00:46,1 days 23:57:54
2,2017-02-09 00:03:06,2017-02-11 00:00:53,2017-03-03 01:01:56,0,1970-01-01 00:00:00,2,0,registry,1970-01-01 00:00:00,-17207 days +23:56:54


# Data Cleaning

We found that some users' subscription date is much earlier than the created date, therefore, here we make some data cleaning before analytics part. 

In [73]:
def data_cleaning_avg_life(data):
    return data[data.First_email > data.created_at]

In [74]:
df = data_cleaning_avg_life(df)

(1777, 10)

In [75]:
df.source.value_counts()

giveaway                   339626
checklist                  313118
registry                   148644
email:pregnancy:direct      16436
event                        1099
email:newsletter:widget       312
email:pregnancy:widget        153
0                              86
Name: source, dtype: int64

In [76]:
def group_by(cat):
    print("----------------")
    print cat
    print df[df.source == cat].Life_time.mean()

In [77]:
pd.Series(df.source.unique()).apply(group_by)

----------------
checklist
-9 days +08:45:41.445595
----------------
registry
-91 days +03:24:08.349358
----------------
0
0 days 00:00:00
----------------
giveaway
0 days 00:00:00
----------------
email:pregnancy:direct
0 days 00:00:00
----------------
email:newsletter:widget
0 days 00:00:00
----------------
email:pregnancy:widget
0 days 00:00:00
----------------
event
0 days 00:00:00


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

# Conclusion:

We are able to compute the average lifetime and compare them by different sources. However, here we found some users' first subscription date is earilier than the created date. Therefore, we made some cleaning base on our assumption. And finally we found out the average lifetime is different according to different sources. We may take a closer look at it in next section. 